# GROMACS molecular dynamics demonstration

Molecular dynamics (MD) is a versatile method for simulating the time-dependent behavior of molecular systems. This notebook demonstrates how MD simulations can be run using the GROMACS software to study the self-assembly of lipids. To accelerate the simulations, a coarse-grained (CG) model using the Martini force field is employed. In addition, parallel processing with multiple CPU cores is leveraged to demonstrate the principles of high-performance computing (HPC).

Let's try inserting 50 dipalmitoylphosphatidylcholine (DPPC) lipids into a cubic simulation box with a side length of 6 Å. The resulting system will be stored in a file named `lipids.gro`.

In [136]:
%run config.py

Dropdown(description='Lipid', index=4, options=('CPC', 'DLPC', 'DOPC', 'DPGS', 'DPPC', 'DPPG', 'DPSM', 'DTPC',…

Dropdown(description='Solvent', options=('W', 'EOL', 'OD'), value='W')

IntSlider(value=50, description='# of lipids', min=10)

FloatSlider(value=6.0, description='Box size', max=10.0, min=5.0, step=0.5)

In [137]:
!bash init_files.sh {lipid.value}

Initialized input files for DPPC (topol.top, em.mdp, md.mdp)


In [138]:
!gmx_mpi insert-molecules -o lipids -ci ff22/{lipid.value}.gro -nmol {nlip.value} -try 200 -box {box.value}

                 :-) GROMACS - gmx insert-molecules, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Tee

Next, we need to fill the `lipids.gro` system with water (solvate). The resulting system will be stored in the file `solv.gro`.

In [139]:
!gmx_mpi solvate -cp lipids.gro -cs ff22/{solvent.value}.gro -o solv # -scale 2.0

                     :-) GROMACS - gmx solvate, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

Check the previous outputs for the number of lipids and water molecules that were *de facto* added. We need to modify a template topology file with these amounts, respectively.

In [140]:
!echo {lipid.value} 50 >> topol.top
!echo {solvent.value} 1792 >> topol.top

Now, let's copy a template recipe for performing *energy minimization*

In [141]:
!gmx_mpi grompp -f em -c solv -p topol -o em -maxwarn 10

                      :-) GROMACS - gmx grompp, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

In [142]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -v -s em

                      :-) GROMACS - gmx mdrun, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

Step=  141, Dmax= 6.6e-02 nm, Epot= -5.30894e+04 Fmax= 8.51725e+02, atom= 1390
Step=  142, Dmax= 7.9e-02 nm, Epot= -5.31308e+04 Fmax= 1.84121e+03, atom= 1440
Step=  143, Dmax= 9.5e-02 nm, Epot= -5.31564e+04 Fmax= 3.46273e+03, atom= 1440
Step=  144, Dmax= 1.1e-01 nm, Epot= -5.33163e+04 Fmax= 1.22611e+03, atom= 1440
Step=  146, Dmax= 6.8e-02 nm, Epot= -5.34187e+04 Fmax= 8.36778e+02, atom= 9270
Step=  147, Dmax= 8.2e-02 nm, Epot= -5.34533e+04 Fmax= 1.63276e+03, atom= 785
Step=  149, Dmax= 4.9e-02 nm, Epot= -5.36043e+04 Fmax= 3.91904e+02, atom= 328
Step=  151, Dmax= 3.0e-02 nm, Epot= -5.36836e+04 Fmax= 6.64570e+02, atom= 1827
Step=  152, Dmax= 3.5e-02 nm, Epot= -5.37241e+04 Fmax= 1.14490e+03, atom= 297
Step=  153, Dmax= 4.3e-02 nm, Epot= -5.37841e+04 Fmax= 1.05708e+03, atom= 297
Step=  154, Dmax= 5.1e-02 nm, Epot= -5.38102e+04 Fmax= 1.45320e+03, atom= 297
Step=  155, Dmax= 6.1e-02 nm, Epot= -5.38333e+04 Fmax= 2.00775e+03, atom= 1827
Step=  156, Dmax= 7.4e-02 nm, Epot= -5.39001e+04 Fmax= 1.

Step=  304, Dmax= 2.6e-02 nm, Epot= -5.71510e+04 Fmax= 4.47346e+02, atom= 1659
Step=  306, Dmax= 1.6e-02 nm, Epot= -5.71650e+04 Fmax= 1.87937e+02, atom= 1659
Step=  307, Dmax= 1.9e-02 nm, Epot= -5.71770e+04 Fmax= 7.65501e+02, atom= 1659
Step=  308, Dmax= 2.3e-02 nm, Epot= -5.71966e+04 Fmax= 3.36341e+02, atom= 1659
Step=  309, Dmax= 2.7e-02 nm, Epot= -5.71997e+04 Fmax= 9.51321e+02, atom= 1659
Step=  310, Dmax= 3.3e-02 nm, Epot= -5.72191e+04 Fmax= 5.37487e+02, atom= 1659
Step=  312, Dmax= 2.0e-02 nm, Epot= -5.72346e+04 Fmax= 2.89685e+02, atom= 1595
Step=  313, Dmax= 2.3e-02 nm, Epot= -5.72420e+04 Fmax= 8.42481e+02, atom= 1659
Step=  314, Dmax= 2.8e-02 nm, Epot= -5.72582e+04 Fmax= 4.61109e+02, atom= 1659
Step=  316, Dmax= 1.7e-02 nm, Epot= -5.72718e+04 Fmax= 2.05997e+02, atom= 1659
Step=  317, Dmax= 2.0e-02 nm, Epot= -5.72789e+04 Fmax= 8.28255e+02, atom= 1659
Step=  318, Dmax= 2.4e-02 nm, Epot= -5.73000e+04 Fmax= 3.62658e+02, atom= 1659
Step=  320, Dmax= 1.5e-02 nm, Epot= -5.73110e+04 Fma

Step=  472, Dmax= 1.1e-02 nm, Epot= -5.82201e+04 Fmax= 2.51239e+02, atom= 1595
Step=  473, Dmax= 1.3e-02 nm, Epot= -5.82237e+04 Fmax= 3.36888e+02, atom= 1595
Step=  474, Dmax= 1.6e-02 nm, Epot= -5.82272e+04 Fmax= 3.65762e+02, atom= 1595
Step=  475, Dmax= 1.9e-02 nm, Epot= -5.82295e+04 Fmax= 4.88779e+02, atom= 1595
Step=  476, Dmax= 2.2e-02 nm, Epot= -5.82322e+04 Fmax= 5.19299e+02, atom= 1595
Step=  478, Dmax= 1.3e-02 nm, Epot= -5.82413e+04 Fmax= 9.20918e+01, atom= 2287
Step=  479, Dmax= 1.6e-02 nm, Epot= -5.82495e+04 Fmax= 5.60655e+02, atom= 1595
Step=  480, Dmax= 1.9e-02 nm, Epot= -5.82564e+04 Fmax= 4.20741e+02, atom= 1595
Step=  482, Dmax= 1.2e-02 nm, Epot= -5.82621e+04 Fmax= 1.14267e+02, atom= 1659
Step=  483, Dmax= 1.4e-02 nm, Epot= -5.82661e+04 Fmax= 5.61246e+02, atom= 1595
Step=  484, Dmax= 1.7e-02 nm, Epot= -5.82739e+04 Fmax= 2.52398e+02, atom= 1595
Step=  486, Dmax= 1.0e-02 nm, Epot= -5.82781e+04 Fmax= 1.69494e+02, atom= 1595
Step=  487, Dmax= 1.2e-02 nm, Epot= -5.82812e+04 Fma

In [143]:
!gmx_mpi make_ndx -f confout.gro < template/ndx_inp

                     :-) GROMACS - gmx make_ndx, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu M

In [144]:
!gmx_mpi grompp -f md -c confout -p topol -maxwarn 10 -n index -o md

                      :-) GROMACS - gmx grompp, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mu

In [145]:
!orterun -n $SLURM_CPUS_PER_TASK --oversubscribe gmx_mpi mdrun -deffnm md -v

                      :-) GROMACS - gmx mdrun, 2021.5 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Teemu Mur

imb F  5% step 8700, remaining wall clock time:    18 s          
imb F  2% step 8800, remaining wall clock time:    18 s          
imb F  2% step 8900, remaining wall clock time:    18 s          
imb F  4% step 9000, remaining wall clock time:    18 s          
imb F  6% step 9100, remaining wall clock time:    18 s          
imb F  4% step 9200, remaining wall clock time:    18 s          
imb F  6% step 9300, remaining wall clock time:    18 s          
imb F  7% step 9400, remaining wall clock time:    18 s          
imb F  6% step 9500, remaining wall clock time:    18 s          
imb F  6% step 9600, remaining wall clock time:    18 s          
imb F  4% step 9700, remaining wall clock time:    18 s          
imb F  5% step 9800, remaining wall clock time:    18 s          
imb F  5% step 9900, remaining wall clock time:    18 s          
imb F  6% step 10000, remaining wall clock time:    18 s          
imb F  6% step 10100, remaining wall clock time:    18 s          
imb F  5

imb F  4% step 21200, remaining wall clock time:    16 s          
imb F  5% step 21300, remaining wall clock time:    16 s          
imb F  7% step 21400, remaining wall clock time:    16 s          
imb F  6% step 21500, remaining wall clock time:    16 s          
imb F  4% step 21600, remaining wall clock time:    16 s          
imb F  6% step 21700, remaining wall clock time:    16 s          
imb F  5% step 21800, remaining wall clock time:    16 s          
imb F  6% step 21900, remaining wall clock time:    16 s          
imb F  6% step 22000, remaining wall clock time:    16 s          
imb F  5% step 22100, remaining wall clock time:    16 s          
imb F  5% step 22200, remaining wall clock time:    16 s          
imb F  5% step 22300, remaining wall clock time:    16 s          
imb F  5% step 22400, remaining wall clock time:    16 s          
imb F  5% step 22500, remaining wall clock time:    16 s          
imb F  4% step 22600, remaining wall clock time:    16 s      

imb F 24% step 33800, remaining wall clock time:    15 s          
imb F  8% step 33900, remaining wall clock time:    15 s          
imb F  7% step 34000, remaining wall clock time:    15 s          
imb F  7% step 34100, remaining wall clock time:    15 s          
imb F 37% step 34200, remaining wall clock time:    15 s          
imb F  7% step 34300, remaining wall clock time:    15 s          
imb F  7% step 34400, remaining wall clock time:    15 s          
imb F  6% step 34500, remaining wall clock time:    15 s          
imb F  7% step 34600, remaining wall clock time:    15 s          
imb F  6% step 34700, remaining wall clock time:    15 s          
imb F  7% step 34800, remaining wall clock time:    15 s          
imb F  5% step 34900, remaining wall clock time:    15 s          
imb F 10% step 35000, remaining wall clock time:    15 s          
imb F  7% step 35100, remaining wall clock time:    15 s          
imb F  9% step 35200, remaining wall clock time:    15 s      

imb F  4% step 46100, remaining wall clock time:    13 s          
imb F  6% step 46200, remaining wall clock time:    13 s          
imb F  6% step 46300, remaining wall clock time:    13 s          
imb F  6% step 46400, remaining wall clock time:    13 s          
imb F  6% step 46500, remaining wall clock time:    13 s          
imb F  5% step 46600, remaining wall clock time:    13 s          
imb F  4% step 46700, remaining wall clock time:    13 s          
imb F  4% step 46800, remaining wall clock time:    13 s          
imb F  9% step 46900, remaining wall clock time:    13 s          
imb F  3% step 47000, remaining wall clock time:    13 s          
imb F  5% step 47100, remaining wall clock time:    13 s          
imb F  5% step 47200, remaining wall clock time:    13 s          
imb F  6% step 47300, remaining wall clock time:    13 s          
imb F  4% step 47400, remaining wall clock time:    13 s          
imb F  7% step 47500, remaining wall clock time:    13 s      

...

imb F  7% step 96100, remaining wall clock time:     6 s          
imb F  5% step 96200, remaining wall clock time:     6 s          
imb F  5% step 96300, remaining wall clock time:     6 s          
imb F  7% step 96400, remaining wall clock time:     6 s          
imb F  6% step 96500, remaining wall clock time:     6 s          
imb F  5% step 96600, remaining wall clock time:     6 s          
imb F  4% step 96700, remaining wall clock time:     6 s          
imb F  7% step 96800, remaining wall clock time:     6 s          
imb F  6% step 96900, remaining wall clock time:     6 s          
imb F  5% step 97000, remaining wall clock time:     6 s          
imb F  6% step 97100, remaining wall clock time:     6 s          
imb F  8% step 97200, remaining wall clock time:     6 s          
imb F  6% step 97300, remaining wall clock time:     6 s          
imb F  6% step 97400, remaining wall clock time:     6 s          
imb F  6% step 97500, remaining wall clock time:     6 s      

imb F  8% step 108200, remaining wall clock time:     5 s          
imb F 10% step 108300, remaining wall clock time:     5 s          
imb F 10% step 108400, remaining wall clock time:     5 s          
imb F 10% step 108500, remaining wall clock time:     5 s          
imb F  7% step 108600, remaining wall clock time:     5 s          
imb F  8% step 108700, remaining wall clock time:     5 s          
imb F  9% step 108800, remaining wall clock time:     5 s          
imb F  9% step 108900, remaining wall clock time:     5 s          
imb F  8% step 109000, remaining wall clock time:     5 s          
imb F  8% step 109100, remaining wall clock time:     5 s          
imb F  6% step 109200, remaining wall clock time:     5 s          
imb F  7% step 109300, remaining wall clock time:     5 s          
imb F  8% step 109400, remaining wall clock time:     5 s          
imb F 10% step 109500, remaining wall clock time:     4 s          
imb F 10% step 109600, remaining wall clock time

imb F  9% step 120300, remaining wall clock time:     3 s          
imb F  9% step 120400, remaining wall clock time:     3 s          
imb F  7% step 120500, remaining wall clock time:     3 s          
imb F  8% step 120600, remaining wall clock time:     3 s          
imb F 10% step 120700, remaining wall clock time:     3 s          
imb F  8% step 120800, remaining wall clock time:     3 s          
imb F  9% step 120900, remaining wall clock time:     3 s          
imb F 10% step 121000, remaining wall clock time:     3 s          
imb F  7% step 121100, remaining wall clock time:     3 s          
imb F  8% step 121200, remaining wall clock time:     3 s          
imb F  9% step 121300, remaining wall clock time:     3 s          
imb F  7% step 121400, remaining wall clock time:     3 s          
imb F 12% step 121500, remaining wall clock time:     3 s          
imb F  8% step 121600, remaining wall clock time:     3 s          
imb F  7% step 121700, remaining wall clock time

imb F 12% step 132500, remaining wall clock time:     2 s          
imb F 11% step 132600, remaining wall clock time:     2 s          
imb F 10% step 132700, remaining wall clock time:     2 s          
imb F 12% step 132800, remaining wall clock time:     2 s          
imb F  7% step 132900, remaining wall clock time:     2 s          
imb F  9% step 133000, remaining wall clock time:     2 s          
imb F  7% step 133100, remaining wall clock time:     2 s          
imb F 12% step 133200, remaining wall clock time:     2 s          
imb F  9% step 133300, remaining wall clock time:     2 s          
imb F 12% step 133400, remaining wall clock time:     2 s          
imb F 10% step 133500, remaining wall clock time:     2 s          
imb F  9% step 133600, remaining wall clock time:     2 s          
imb F  8% step 133700, remaining wall clock time:     2 s          
imb F  9% step 133800, remaining wall clock time:     2 s          
imb F  8% step 133900, remaining wall clock time

imb F  9% step 144700, remaining wall clock time:     0 s          
imb F  9% step 144800, remaining wall clock time:     0 s          
imb F  9% step 144900, remaining wall clock time:     0 s          
imb F 13% step 145000, remaining wall clock time:     0 s          
imb F  8% step 145100, remaining wall clock time:     0 s          
imb F 10% step 145200, remaining wall clock time:     0 s          
imb F  5% step 145300, remaining wall clock time:     0 s          
imb F  9% step 145400, remaining wall clock time:     0 s          
imb F  7% step 145500, remaining wall clock time:     0 s          
imb F  8% step 145600, remaining wall clock time:     0 s          
imb F 10% step 145700, remaining wall clock time:     0 s          
imb F  9% step 145800, remaining wall clock time:     0 s          
imb F 10% step 145900, remaining wall clock time:     0 s          
imb F  9% step 146000, remaining wall clock time:     0 s          
imb F 11% step 146100, remaining wall clock time

In [ ]:
!gmx_mpi trjconv -s md -f md.xtc -o md_whole.xtc -pbc mol < template/trj_inp

In [ ]:
u = mda.Universe('md.gro', 'md_whole.xtc')
view = nv.show_mdanalysis(u)
view.add_spacefill(lipid.value)
view.add_point(solvent.value)
view.add_unitcell()
view

Cleanup

In [ ]:
!bash clean.sh